<a href="https://colab.research.google.com/github/paritoshkc/TextAnalytics/blob/master/Review_Scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import lxml.html as lh
import pandas as pd
import re



# Get movie name and year list from wikipedia 
url_wiki='https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films'
#Create a handle, page, to handle the contents of the website
page = requests.get(url_wiki).text
soup= BeautifulSoup(page, "lxml")

My_table = soup.find('table',{'class':'wikitable sortable'})
links = My_table.find_all('a')

# for future use
name=[]
year=[]


# used for constrcution of urls
url_front = 'https://www.rottentomatoes.com/m/'
url_back_critics='/reviews'
url_back_audi='/reviews?type=user'

url_critics=[]
url_audi=[]
url_mid=[]


i=0
for link in links:
  texts.append(link.text.split('\n'))
for text in texts:
  if text[0]=='[1]':
    continue
  if i%2==0:
    text_string=text[0].replace(' ','_').lower()
    name.append(text_string)
  else:
    year.append(text[0])
    if int(text[0])<2018:
      break
  i+=1

zippedList =  list(zip(name,year))

for n,y in zippedList:
  mid=url_mid.append(n+'_'+y)

for mid in url_mid:
  url_critics.append(url_front+mid+url_back_critics)
  url_audi.append(url_front+mid+url_back_audi)

zippedList =  list(zip(url_mid,url_critics,url_audi))

# for future use
dfObj = pd.DataFrame(zippedList, columns = ['Name' , 'url_critics','url_audi']) 




In [0]:

def get_user_reviews():
    
  for url in url_audi:
    if '2019' in url:
      print(url)
      response = requests.get(url)

      soup = BeautifulSoup(response.text, "html.parser")


      # audience reviews
      audience_reviews=soup.findAll('p', attrs={'class': 'audience-reviews__review--mobile js-review-text clamp clamp-4 js-clamp'})
      if len(audience_reviews)!=0 :
        for reviews in audience_reviews:
          print(reviews.text)
      

In [0]:
def get_ctric_reviews():

  review_pat = re.compile(r'<div class=\"the_review\">[;a-zA-Z\s,-.\'\/\?\[\]\":\']*</div>')


  for url in url_critics:
    if '2019' in url:
      print(url)
      response = requests.get(url)
      soup = BeautifulSoup(response.text, "html.parser")

      soup = str(soup)
      if not len(soup.split('="review_table'))==1:
        
        review_soup = soup.split('="review_table')[1].split('row review_table_row')
        review_soup.pop(0)
        review_soup = soup.split('="review_table')[1].split('row review_table_row')
        review_soup.pop(0)
        for review in review_soup:
                
                # extract review
                match = re.findall(review_pat, str(review))
                if len(match) > 0:
                    m = match[0]
                    for iden in ['<div class="the_review"> ','</div>']:
                        m = m.replace(iden,'')
                    reviews.append(m.strip('"'))
            

        # if len(audience_reviews)!=0 :
        for review in reviews:
          if review.startswith('<div class="the_review">'):
            print(review.replace('<div class="the_review">','').replace('                                   ','').replace('\n',''))


In [166]:
get_ctric_reviews()

Streaming output truncated to the last 5000 lines.
 Ultimately, Joker blames society for its "hero's" unraveling, letting this maniac off the hook. Simplistic, yes. Reckless, undoubtedly. But not totally invalid.                                
 ...A lethally dull and spiritually deadening exercise whose strained black humor and self-importance makes even the profundities of Christopher Nolan's Dark Knight trilogy feel meaningful by comparison.                                
 I implore you: move on from Joker. You are better than this. We all are.                                
 His story wasn't interesting to me because there never was a valid option for him to chose a different path.                                
 Is it good? Parts of it. Did I enjoy it? Not really.                                
 Todd Phillips creates a terrifying masterpiece and one of the best and most daring comic book films ever made.                                
 To celebrate "Joker" as the best of the 

In [164]:
get_user_reviews()

https://www.rottentomatoes.com/m/parasite_2019/reviews?type=user
This was the worst movie I have seen in a long time. Do not waste your money on seeing it. When the movie was over my wife and I sat there almost in shock and we both said, "How could this have been given the Oscar for the Best Movie of the Year? Just pass on this one and go see something else; anything else!
（9.0）Interesting！电影《寄生虫》很有趣，富人家地下室那段开始变得非常精彩。金基泽一家和雯光一家都代表的是穷人家庭，甚至男主人曾经开的都是台湾古早味蛋糕店，（而且最后都失败了）但他们两家人还是展开斗争，最后一把刀酿成三家人的悲剧。浮在水面上的神石，小儿子抽象的画作，难闻的气味……这些线索推动或暗示故事发展，最后发生了一个略荒诞的故事。
A beautifully photographed and acted,  fearless and fantastical exploration of class tension, poverty, entitlement, human nature, gullibility, and primal urges.  A tour de force deserving of best picture.
This movie traumatized my father, then it traumatized me. Why me. Thank you based Korea.
It was shot well but apart from that basic and boring
Highly overrated but worth a look 👍
Masterpiece! No way this movie can be pulled off in the United S